# Profile hydrologic model

In [1]:
import numpy as np
from numpy.typing import NDArray
import pandas as pd 
from pandas import DataFrame, Series
import potions as pt

In [2]:
data_path: str = "../input/Sleepers_Results.txt"
in_df: DataFrame = pd.read_csv(data_path, sep="\\s+", index_col="Date", parse_dates=True)

In [3]:
params: pt.HbvParameters = pt.HbvParameters.default()
init_state: NDArray = np.array([1.0, 10.0, 10.0, 25.0])
dates: Series = in_df.index # type: ignore
forc: pt.ForcingData = pt.ForcingData(
    precip=in_df.Precipitation,
    temp=in_df.Temperature,
    pet=in_df.PET
)

In [6]:
%%prun
res = pt.run_hbv_model(params, init_state, forc, dates=dates, verbose=False)

         343299 function calls (343221 primitive calls) in 0.265 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     2231    0.039    0.000    0.108    0.000 rk.py:14(rk_step)
    15576    0.027    0.000    0.066    0.000 hydro.py:98(mass_balance)
      2/1    0.025    0.013    0.001    0.001 events.py:87(_run)
     2229    0.014    0.000    0.153    0.000 rk.py:111(_step_impl)
    21138    0.010    0.000    0.010    0.000 {built-in method builtins.min}
     5516    0.009    0.000    0.018    0.000 _linalg.py:2575(norm)
    11178    0.008    0.000    0.008    0.000 hydro.py:334(lat_flux)
     1095    0.007    0.000    0.028    0.000 common.py:68(select_initial_step)
     1095    0.007    0.000    0.234    0.000 ivp.py:159(solve_ivp)
    15576    0.006    0.000    0.078    0.000 base.py:22(fun_wrapped)
     5493    0.006    0.000    0.009    0.000 hydro.py:290(lat_flux)
     5493    0.005    0.000    0.009    0.000 hydro.py

,s_snow_0,q_forc_snow_0,q_vap_snow_0,q_lat_snow_0,q_vert_snow_0,s_soil_1,q_forc_soil_1,q_vap_soil_1,q_lat_soil_1,q_vert_soil_1,s_ground_2,q_forc_ground_2,q_vap_ground_2,q_lat_ground_2,q_vert_ground_2,s_ground_3,q_forc_ground_3,q_vap_ground_3,q_lat_ground_3,q_vert_ground_3
Date,,,,,,,,,,,,,,,,,,,,
2016-10-01,0.0,0.0,0.0,0.0,1.0,10.727458,0.0,0.243513,0.072746,0.000000,8.990000,0.0,0.0,0.01,1.0,25.99900,0.0,0.0,0.001,0.0
2016-10-02,0.0,0.0,0.0,0.0,0.0,10.488705,0.0,0.176210,0.048871,0.000000,7.980000,0.0,0.0,0.01,1.0,26.99800,0.0,0.0,0.001,0.0
2016-10-03,0.0,0.0,0.0,0.0,0.0,13.811870,4.3,0.263807,0.381187,0.593910,7.563910,0.0,0.0,0.01,1.0,27.99700,0.0,0.0,0.001,0.0
2016-10-04,0.0,0.0,0.0,0.0,0.0,13.205078,0.0,0.250896,0.320508,0.000000,6.553910,0.0,0.0,0.01,1.0,28.99600,0.0,0.0,0.001,0.0
2016-10-05,0.0,0.0,0.0,0.0,0.0,12.617716,0.0,0.290207,0.261772,0.000000,5.543910,0.0,0.0,0.01,1.0,29.99500,0.0,0.0,0.001,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-26,0.0,0.0,0.0,0.0,0.0,9.803108,0.0,0.384282,0.000000,0.000000,180.867638,0.0,0.0,0.01,1.0,314.53798,0.0,0.0,0.001,0.0
2017-09-27,0.0,0.0,0.0,0.0,0.0,11.145175,2.0,0.422402,0.114517,0.222903,180.080541,0.0,0.0,0.01,1.0,315.53698,0.0,0.0,0.001,0.0
2017-09-28,0.0,0.0,0.0,0.0,0.0,11.813919,1.5,0.524538,0.181392,0.177209,179.247750,0.0,0.0,0.01,1.0,316.53598,0.0,0.0,0.001,0.0
